In [1]:
# Importing the model building libraries
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
train = ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2, horizontal_flip=True)
test = ImageDataGenerator(rescale=1./255)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4qJAQb7dwGLC-B9-LTrVWLg5NFiS1cHQz-e2sUHBAtzr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'emergingmethodsforearlydetectiono-donotdelete-pr-kxvncfzuancknt'
object_key = 'dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
train_dataset = train.flow_from_directory('./forest_fire/train',
                                          target_size=(64,64),
                                          batch_size=32,
                                          class_mode='binary')
test_dataset = test.flow_from_directory('./forest_fire/test',
                                        target_size=(64,64),
                                        batch_size=32,
                                        class_mode='binary')

Found 1832 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


In [6]:
test_dataset.class_indices

{'fire': 0, 'nofire': 1}

In [7]:
# Initializing the model
model = keras.Sequential()

# Adding CNN layers
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())

# Adding Dense layers
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [8]:
# Configuring the learning process
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [9]:
# Training the model
r=model.fit(train_dataset,epochs=8,validation_data=test_dataset)

Epoch 1/8
58/58 [==============================] - 266s 5s/step - loss: 0.3363 - accuracy: 0.8581 - val_loss: 0.2938 - val_accuracy: 0.9118
Epoch 2/8
58/58 [==============================] - 264s 5s/step - loss: 0.1915 - accuracy: 0.9410 - val_loss: 0.4224 - val_accuracy: 0.8235
Epoch 3/8
58/58 [==============================] - 163s 3s/step - loss: 0.1706 - accuracy: 0.9432 - val_loss: 0.3413 - val_accuracy: 0.8529
Epoch 4/8
58/58 [==============================] - 84s 1s/step - loss: 0.1658 - accuracy: 0.9481 - val_loss: 0.5765 - val_accuracy: 0.7941
Epoch 5/8
58/58 [==============================] - 86s 1s/step - loss: 0.1754 - accuracy: 0.9394 - val_loss: 0.3683 - val_accuracy: 0.7794
Epoch 6/8
58/58 [==============================] - 86s 1s/step - loss: 0.1341 - accuracy: 0.9552 - val_loss: 0.3361 - val_accuracy: 0.9118
Epoch 7/8
58/58 [==============================] - 85s 1s/step - loss: 0.1258 - accuracy: 0.9569 - val_loss: 0.2917 - val_accuracy: 0.9118
Epoch 8/8
58/58 [=======

In [10]:
# Saving the model
model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)
model.save_weights('model.h5')

In [12]:
# Predictions
predictions = model.predict(test_dataset)
predictions = np.round(predictions)

In [13]:
!tar -zcvf model.tgz model.h5

model.h5


In [14]:
ls -1

forest_fire/
model.h5
model.json
model.tgz


In [15]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 11.3 MB/s eta 0:00:01


In [16]:
!pip install ibm_watson_machine_learning

In [18]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"vDiEyzT4r7l7stZn8RvvznduedNtpantsJxhzjzpdl4X"
}
client = APIClient(wml_credentials)
client

In [19]:
client.spaces.get_details()

{'resources': [{'entity': {'description': '',
    'name': 'Emerging methods for early detection of forest fire',
    'scope': {'bss_account_id': 'f7575aaff2d749edafe54eadff87ea0a'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': 'dd1eaf0d-2bd6-4e15-adc6-ca67b5be66e8',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '298bbcb01afe46cdbd1197bc66d5c419',
        'api_key': 'nMTc_cryhjg48W8dsLEvIhHE7eOFIfoUY4tM6hA4r8R0',
        'secret_access_key': 'fa63fb7d787fab5ec5438d53a33bbd64fe434553343d04dd',
        'service_id': 'ServiceId-a3cc86fc-3700-4bcf-b4d0-fd54a5209191'},
       'editor': {'access_key_id': '12e35dc642684d2d8a47e6b9a5663ff0',
        'api_key': 'Va8JOXThs7X6RU5emITQNsvUGm4ZVOGns6b355P84Ufy',
        'resource_key_crn': 'crn:v1:bluemix:public:cloud-object-storage:global:a/f7575aaff2d749edafe54eadff87ea0a:eedf8410-437b-44ea-970c-b77c2d3b4d9b::',
        'secret_access_key'

In [20]:
space_id = 'b0616d3b-aefd-4fdb-b530-2dde14ea1f42'

In [21]:
client.set.default_space(space_id)

'SUCCESS'

In [22]:
client.software_specifications.list()


-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [23]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [24]:
model_details = client.repository.store_model(model='model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME : 'Forest fire detection model',
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-18T13:09:07.581Z',
  'id': 'f0facc79-ffab-4523-854b-48b527ffa429',
  'modified_at': '2022-11-18T13:09:10.089Z',
  'name': 'Forest fire detection model',
  'owner': 'IBMid-661004479I',
  'resource_key': '02925dfe-e1fd-4141-989d-8df72e5e6f80',
  'space_id': 'b0616d3b-aefd-4fdb-b530-2dde14ea1f42'},
 'system': {'warnings': []}}

In [25]:
model_id = client.repository.get_model_id(model_details)
model_id

'f0facc79-ffab-4523-854b-48b527ffa429'

In [26]:
client.repository.download(model_id,'model.tar.gb')

Successfully saved model content to file: 'model.tar.gb'


'/home/wsuser/work/model.tar.gb'

In [27]:
ls -1

forest_fire/
model.h5
model.json
model.tar.gb
model.tgz
